In [24]:
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from scipy.stats import ttest_ind, chi2_contingency
from openai import OpenAI
my_key = os.getenv('CHAT_GPT_KEY')
apikey = my_key


In [2]:
# Load the Parquet file into a DataFrame
df = pd.read_parquet('final_df.parquet')

print(df.head())  
print(df.columns) 

      Attrition_Flag  Customer_Age Gender  Dependent_count Education_Level  \
0  Existing Customer            45      M                3     High School   
1  Existing Customer            49      F                5        Graduate   
2  Existing Customer            51      M                3        Graduate   
3  Existing Customer            40      F                4     High School   
4  Existing Customer            40      M                3      Uneducated   

  Marital_Status Income_Category Card_Category  Months_on_book  \
0        Married     $60K - $80K          Blue              39   
1         Single  Less than $40K          Blue              44   
2        Married    $80K - $120K          Blue              36   
3        Unknown  Less than $40K          Blue              34   
4        Married     $60K - $80K          Blue              21   

   Total_Relationship_Count  Months_Inactive_12_mon  Contacts_Count_12_mon  \
0                         5                       1     

In [42]:
df['Attrition_Flag'].unique()

array(['Existing Customer', 'Attrited Customer'], dtype=object)

In [3]:
# T-test for numerical features between attrited and existing customers
p_values = {}
for feature in df.select_dtypes(include='number').columns:
    group1 = df[df['Attrition_Flag'] == 'Attrited Customer'][feature]
    group2 = df[df['Attrition_Flag'] == 'Existing Customer'][feature]
    t_stat, p_value = ttest_ind(group1, group2)
    p_values[feature] = p_value
    print(f"{feature}: t-stat={t_stat:.2f}, p-value={p_value:.4f}")

Customer_Age: t-stat=1.83, p-value=0.0670
Dependent_count: t-stat=1.91, p-value=0.0560
Months_on_book: t-stat=1.38, p-value=0.1684
Total_Relationship_Count: t-stat=-15.27, p-value=0.0000
Months_Inactive_12_mon: t-stat=15.52, p-value=0.0000
Contacts_Count_12_mon: t-stat=21.02, p-value=0.0000
Total_Revolving_Bal: t-stat=-27.44, p-value=0.0000
Total_Trans_Ct: t-stat=-40.25, p-value=0.0000
Total_Trans_Amt_BoxCox: t-stat=-24.10, p-value=0.0000
Total_Ct_Chng_Q4_Q1_BoxCox: t-stat=-36.95, p-value=0.0000
Total_Amt_Chng_Q4_Q1_BoxCox: t-stat=-16.19, p-value=0.0000
Avg_Utilization_Ratio_BoxCox: t-stat=-23.00, p-value=0.0000
Credit_Limit_BoxCox: t-stat=-5.32, p-value=0.0000
Avg_Open_To_Buy_BoxCox: t-stat=1.68, p-value=0.0924


In [7]:
categorical_features = ['Gender', 'Marital_Status', 'Income_Category', 'Card_Category']

# Chi-square test for categorical features
for feature in categorical_features:
    contingency_table = pd.crosstab(df[feature], df['Attrition_Flag'])
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    print(f"{feature}: chi2={chi2:.2f}, p-value={p_value:.4f}")

Gender: chi2=13.87, p-value=0.0002
Marital_Status: chi2=6.06, p-value=0.1089
Income_Category: chi2=12.83, p-value=0.0250
Card_Category: chi2=2.23, p-value=0.5252


In [33]:
import pandas as pd

# Assuming df is your DataFrame
df_encoded = pd.get_dummies(df, columns=['Attrition_Flag', 'Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category'])

# Compute the correlation matrix
correlation_matrix = df_encoded.corr()

# Filter the correlation matrix to only include columns related to 'Attrition_Flag'
attrition_columns = [col for col in correlation_matrix.columns if 'Attrition_Flag' in col]

# Get correlations between 'Attrition_Flag' columns and all other columns
correlation_attrition = correlation_matrix[attrition_columns]

# Print the filtered correlation matrix
print(correlation_attrition)


                                  Attrition_Flag_Attrited Customer  \
Customer_Age                                              0.018203   
Dependent_count                                           0.018991   
Months_on_book                                            0.013687   
Total_Relationship_Count                                 -0.150005   
Months_Inactive_12_mon                                    0.152449   
Contacts_Count_12_mon                                     0.204491   
Total_Revolving_Bal                                      -0.263053   
Total_Trans_Ct                                           -0.371403   
Total_Trans_Amt_BoxCox                                   -0.232896   
Total_Ct_Chng_Q4_Q1_BoxCox                               -0.344669   
Total_Amt_Chng_Q4_Q1_BoxCox                              -0.158890   
Avg_Utilization_Ratio_BoxCox                             -0.222856   
Credit_Limit_BoxCox                                      -0.052821   
Avg_Open_To_Buy_BoxC

In [35]:
# Save the correlation matrix to a text file
with open('correlation_attrition.txt', 'w') as f:
    f.write(str(correlation_attrition))

print("Correlation matrix saved to 'correlation_attrition.txt'")


Correlation matrix saved to 'correlation_attrition.txt'


In [45]:
def calculate_attrition_percentages(df):
    attrited = df[df['Attrition_Flag'] == 'Attrited Customer']
    total_attrited = len(attrited)
    
    percentages = {}
    
    for column in df.columns:
        if df[column].dtype in ['int64', 'float64']:
            # For numerical columns, create bins
            bins = pd.cut(df[column], bins=5)  # You can adjust the number of bins
            percentages[column] = (attrited[column].groupby(bins).size() / total_attrited * 100).to_dict()
        elif df[column].dtype == 'object':
            # For categorical columns
            percentages[column] = (attrited[column].value_counts() / total_attrited * 100).to_dict()
    
    return percentages

# Calculate percentages
attrition_percentages = calculate_attrition_percentages(df)

# Function to format the percentage information
def format_percentage_info(column, percentages):
    info = "Attrition Percentages:\n"
    for category, percentage in percentages[column].items():
        info += f"  - {category}: {percentage:.2f}%\n"
    return info

/var/folders/tn/bllk4s_n0k9g75m59d6nyp8w0000gn/T/ipykernel_57468/171616175.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentages[column] = (attrited[column].groupby(bins).size() / total_attrited * 100).to_dict()
/var/folders/tn/bllk4s_n0k9g75m59d6nyp8w0000gn/T/ipykernel_57468/171616175.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentages[column] = (attrited[column].groupby(bins).size() / total_attrited * 100).to_dict()
/var/folders/tn/bllk4s_n0k9g75m59d6nyp8w0000gn/T/ipykernel_57468/171616175.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True i

In [47]:
# Prepare the data summary
data_summary = """

# Customer Attrition Analysis: Data Summary

## Significant Numerical Features (p-value < 0.05)

# Customer Attrition Analysis: Data Summary

## Significant Numerical Features (p-value < 0.05)

### Customer Engagement Metrics
- **Total_Relationship_Count**: t-stat=-15.27, p-value=0.0000  
  *Definition*: The total number of products or services a customer has with the bank.  
  *Interpretation*: A lower count is associated with higher attrition risk.
  {total_relationship_count_percentages}

- **Months_Inactive_12_mon**: t-stat=15.52, p-value=0.0000  
  *Definition*: The number of months in the last 12 months during which the customer had no activity.  
  *Interpretation*: More months of inactivity correlate with a higher likelihood of attrition.
  {months_inactive_percentages}

- **Contacts_Count_12_mon**: t-stat=21.02, p-value=0.0000  
  *Definition*: The total number of times the customer contacted the bank in the last 12 months.  
  *Interpretation*: Increased contacts may indicate engagement, reducing churn risk.
  {contacts_count_percentages}

### Financial Metrics
- **Credit_Limit**: t-stat=-2.40, p-value=0.0163  
  *Definition*: The maximum credit limit assigned to the customer's credit card(s).  
  *Interpretation*: Lower credit limits are associated with higher attrition risk.
  {credit_limit_percentages}

- **Total_Revolving_Bal**: t-stat=-27.44, p-value=0.0000  
  *Definition*: The total outstanding balance on revolving credit accounts (e.g., credit cards).  
  *Interpretation*: Higher balances are linked to increased churn likelihood.
  {total_revolving_bal_percentages}

- **Avg_Utilization_Ratio**: t-stat=-18.24, p-value=0.0000  
  *Definition*: The average proportion of credit used relative to the total credit limit across all accounts.  
  *Interpretation*: Higher utilization ratios are associated with greater attrition risk.
  {avg_utilization_ratio_percentages}

### Transaction Patterns
- **Total_Trans_Amt**: t-stat=-17.21, p-value=0.0000  
  *Definition*: The total amount of transactions made by the customer in the last 12 months.  
  *Interpretation*: A lower transaction amount is linked to higher attrition rates.
  {total_trans_amt_percentages}

- **Total_Trans_Ct**: t-stat=-40.25, p-value=0.0000  
  *Definition*: The total number of transactions conducted by the customer in the last 12 months.  
  *Interpretation*: Fewer transactions correlate with increased likelihood of churn.
  {total_trans_ct_percentages}

- **Total_Amt_Chng_Q4_Q1**: t-stat=-13.30, p-value=0.0000  
  *Definition*: The change in transaction amount from Q4 to Q1, indicating seasonal spending behavior.  
  *Interpretation*: A negative change may suggest declining engagement.
  {total_amt_chng_q4_q1_percentages}

- **Total_Ct_Chng_Q4_Q1**: t-stat=-30.50, p-value=0.0000  
  *Definition*: The change in transaction count from Q4 to Q1, reflecting shifts in customer activity levels.  
  *Interpretation*: Fewer transactions in Q1 compared to Q4 are linked to higher attrition risk.
  {total_ct_chng_q4_q1_percentages}

## Significant Categorical Features

### Demographic Factors
- **Gender**: chi2=13.87, p-value=0.0002  
  *Definition*: The gender of the customer (Male/Female/Other).  
  *Interpretation*: Gender differences may influence attrition patterns.
  {gender_percentages}

- **Income_Category**: chi2=12.83, p-value=0.0250  
  *Definition*: The income bracket of the customer (e.g., Less than $40K, $40K - $60K).  
  *Interpretation*: Different income levels may correlate with varying churn risks.
  {income_category_percentages}


### Correlations 

                                  Attrition_Flag_Attrited Customer  \
Customer_Age                                              0.018203   
Dependent_count                                           0.018991   
Months_on_book                                            0.013687   
Total_Relationship_Count                                 -0.150005   
Months_Inactive_12_mon                                    0.152449   
Contacts_Count_12_mon                                     0.204491   
Total_Revolving_Bal                                      -0.263053   
Total_Trans_Ct                                           -0.371403   
Total_Trans_Amt_BoxCox                                   -0.232896   
Total_Ct_Chng_Q4_Q1_BoxCox                               -0.344669   
Total_Amt_Chng_Q4_Q1_BoxCox                              -0.158890   
Avg_Utilization_Ratio_BoxCox                             -0.222856   
Credit_Limit_BoxCox                                      -0.052821   
Avg_Open_To_Buy_BoxCox                                    0.016723   
Attrition_Flag_Attrited Customer                          1.000000   
Attrition_Flag_Existing Customer                         -1.000000   
Gender_F                                                  0.037272   
Gender_M                                                 -0.037272   
Education_Level_College                                  -0.007840   
Education_Level_Doctorate                                 0.029386   
Education_Level_Graduate                                 -0.009046   
Education_Level_High School                              -0.011730   
Education_Level_Post-Graduate                             0.011127   
Education_Level_Uneducated                               -0.001444   
Education_Level_Unknown                                   0.009005   
Marital_Status_Divorced                                   0.000850   
Marital_Status_Married                                   -0.023735   
Marital_Status_Single                                     0.019037   
Marital_Status_Unknown                                    0.008904   
Income_Category_$120K +                                   0.009584   
Income_Category_$40K - $60K                              -0.011688   
Income_Category_$60K - $80K                              -0.028221   
Income_Category_$80K - $120K                             -0.003459   
Income_Category_Less than $40K                            0.022466   
Income_Category_Unknown                                   0.007179   
Card_Category_Blue                                        0.003216   
Card_Category_Gold                                        0.005973   
Card_Category_Platinum                                    0.010823   
Card_Category_Silver                                     -0.008467   

                                  Attrition_Flag_Existing Customer  
Customer_Age                                             -0.018203  
Dependent_count                                          -0.018991  
Months_on_book                                           -0.013687  
Total_Relationship_Count                                  0.150005  
Months_Inactive_12_mon                                   -0.152449  
Contacts_Count_12_mon                                    -0.204491  
Total_Revolving_Bal                                       0.263053  
Total_Trans_Ct                                            0.371403  
Total_Trans_Amt_BoxCox                                    0.232896  
Total_Ct_Chng_Q4_Q1_BoxCox                                0.344669  
Total_Amt_Chng_Q4_Q1_BoxCox                               0.158890  
Avg_Utilization_Ratio_BoxCox                              0.222856  
Credit_Limit_BoxCox                                       0.052821  
Avg_Open_To_Buy_BoxCox                                   -0.016723  
Attrition_Flag_Attrited Customer                         -1.000000  
Attrition_Flag_Existing Customer                          1.000000  
Gender_F                                                 -0.037272  
Gender_M                                                  0.037272  
Education_Level_College                                   0.007840  
Education_Level_Doctorate                                -0.029386  
Education_Level_Graduate                                  0.009046  
Education_Level_High School                               0.011730  
Education_Level_Post-Graduate                            -0.011127  
Education_Level_Uneducated                                0.001444  
Education_Level_Unknown                                  -0.009005  
Marital_Status_Divorced                                  -0.000850  
Marital_Status_Married                                    0.023735  
Marital_Status_Single                                    -0.019037  
Marital_Status_Unknown                                   -0.008904  
Income_Category_$120K +                                  -0.009584  
Income_Category_$40K - $60K                               0.011688  
Income_Category_$60K - $80K                               0.028221  
Income_Category_$80K - $120K                              0.003459  
Income_Category_Less than $40K                           -0.022466  
Income_Category_Unknown                                  -0.007179  
Card_Category_Blue                                       -0.003216  
Card_Category_Gold                                       -0.005973  
Card_Category_Platinum                                   -0.010823  
Card_Category_Silver                                      0.008467  

*Note: All features listed above are statistically significant in relation to customer attrition (p-value < 0.05). Negative t-stats indicate that lower values of the feature are associated with higher attrition risk, while positive t-stats indicate that higher values are associated with higher attrition risk.*



"""

In [58]:
# Create the prompt
cohort_prompt = f"""



Based on the provided {data_summary}, conduct a comprehensive cohort analysis to identify where customer attrition is most severe and where immediate action can yield significant benefits. Follow these steps:

Data Validation
Review the provided data summary and highlight any inconsistencies, missing information, or potential data quality issues that could affect the analysis. State any limitations or assumptions made based on the available data.

Feature Importance Ranking
Rank the significant features based on their statistical importance (using p-values and correlation coefficients) in predicting attrition. Use this ranking to inform cohort definitions.

High-Risk Cohort Identification
Identify 3-4 distinct customer segments with the highest attrition risk. For each cohort:
a) Define the cohort using a combination of at least two significant features. Provide specific thresholds or categories for each feature used.
b) Ensure each cohort captures at least 15% of attrited customers, with no single cohort exceeding 40%.
c) Discuss the characteristics contributing to their high attrition risk, referencing relevant statistical insights.
d) Highlight patterns or trends observed in these cohorts indicating why they may be at risk.
e) State the percentage of total attrited customers captured by each cohort.

Ensure that the defined segments collectively account for at least 80% of attrited customers. If this threshold is not met, provide a second iteration of cohort definitions with explanations for the changes made.

Cross-Validation
For each defined cohort, cross-validate its relevance by examining how it performs across other significant features not used in its definition.

Root Cause Analysis
For each identified high-risk cohort:
a) Analyze the primary drivers of attrition, using specific feature values or trends from the data summary.
b) Identify any notable changes in behavior or engagement metrics leading up to attrition.
c) Discuss any surprising findings that could inform targeted retention strategies.

Quick-Win Retention Strategies
For each high-risk cohort, propose:
a) One actionable retention strategy that can be implemented within 30 days.
b) Describe how this strategy directly addresses the identified risk factors for that cohort.
c) Suggest specific metrics to measure the strategy's effectiveness in reducing attrition.

Long-Term Value Opportunity
Identify one additional customer segment that:
a) Shows moderate attrition risk but has high potential lifetime value.
b) Explain why focusing on this segment could lead to significant long-term benefits for the business.
c) Propose a tailored retention strategy for this segment that balances immediate actions with long-term growth potential.

Action Priority Matrix
Create a 2x2 matrix to prioritize proposed strategies:
a) Plot each strategy based on ease of implementation (x-axis) and potential impact on reducing attrition (y-axis).
b) Use a qualitative scale (High, Medium, Low) for each axis.
c) Briefly explain the rationale for each strategy's positioning on the matrix.

Throughout your analysis:

Ensure all conclusions are supported by specific insights derived from the provided statistical analysis.

Provide data-driven justification for each insight, strategy, or conclusion presented.

Focus on actionable insights that can drive immediate business value.

Avoid making claims without direct evidence from the provided data.

Clearly state any limitations or assumptions in your analysis approach.

Present your analysis in a clear, structured format with appropriate headings for each section.
"""

In [59]:
# Create an OpenAI client
client = OpenAI(api_key=apikey)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a data analyst specializing in cohort analysis."},
        {"role": "user", "content": cohort_prompt}
    ]
)

In [61]:
# Save response to a text file
with open("cohort_analysis_v2.md", "w") as file:
    file.write(response.choices[0].message.content)

# Open and read the file to verify
with open("cohort_analysis_v2.md", "r") as file:
    print(file.read())

# Customer Attrition Cohort Analysis

## 1. Data Validation

### Inconsistencies and Data Quality Issues
1. **Missing Data**: Specific percentages and percentages for customer engagement, trends, and demographic aspects are denoted as placeholders (e.g., `{total_relationship_count_percentages}`). A comprehensive data set should replace these placeholders for analysis.
2. **Contextual Information**: There is no temporal information about when the atrition occurred which can limit our understanding of customer behaviors leading to attrition.
3. **Categorization**: The demographic and categorical data need clarification on how they are structured. For example, knowing the distribution of income categories or education levels would help in further analysis.
4. **Data Staleness**: The analysis is based on data available till October 2023, and any ongoing trends post this date are not considered.

### Limitations and Assumptions
- The analysis assumes that all provided data points are comple